# MNIST example for Menoh Haskell 

In [1]:
{-# LANGUAGE ScopedTypeVariables #-}
import Control.Applicative
import Control.Monad
import System.FilePath
import Text.Printf
import Menoh

In [2]:
Menoh.version

Version {versionBranch = [1,0,2], versionTags = []}

In [3]:
Menoh.bindingVersion

Version {versionBranch = [0,2,0], versionTags = []}

In [4]:
dataDir = "data"

##  Step 1: Prepare input for inference

In [5]:
import qualified Codec.Picture as Picture
import qualified Codec.Picture.Types as Picture

In [6]:
images <- forM [0..9] $ \i -> do
  let fname :: String
      fname = printf "%d.png" i
  ret <- Picture.readImage $ dataDir </> fname
  case ret of
    Left e -> error e
    Right img -> return (Picture.extractLumaPlane $ Picture.convertRGB8 img, i, fname)

In [7]:
[img | (img, _, _) <- images]

## Step 2: Loading ONNX model

In [8]:
batch_size  = length images
channel_num = 1
height = 28
width  = 28
category_num = 10

input_dims, output_dims :: Dims
input_dims  = [batch_size, channel_num, height, width]
output_dims = [batch_size, category_num]

-- Aliases to onnx's node input and output tensor name
mnist_in_name  = "139900320569040"
mnist_out_name = "139898462888656"

In [9]:
-- Load ONNX model data
model_data <- makeModelDataFromONNXFile (dataDir </> "mnist.onnx")

In [10]:
-- Specify inputs and outputs
vpt <- makeVariableProfileTable
         [(mnist_in_name, DTypeFloat, input_dims)]
         [mnist_out_name]
         model_data
optimizeModelData model_data vpt

In [11]:
-- Construct computation primitive list and memories
model <- makeModel vpt model_data "mkldnn"

## Step 3: Run inference

In [12]:
import Data.Word
import qualified Data.Vector.Generic as VG
import qualified Data.Vector.Storable as VS

-- Copy input image data to model's input array
writeBuffer model mnist_in_name [VG.map fromIntegral (Picture.imageData img) :: VS.Vector Float | (img,_,_) <- images]

-- Run inference
run model

-- Get output
(vs :: [VS.Vector Float]) <- readBuffer model mnist_out_name

## Step 4: Examine the results

In [13]:
softmax :: (Real a, Floating a, VG.Vector v a) => v a -> v a
softmax v | VG.null v = VG.empty
softmax v = VG.map (/ s) v'
  where
    m = VG.maximum v
    v' = VG.map (\x -> exp (x - m)) v
    s = VG.sum v'

In [14]:
forM_ (zip images vs) $ \((img,expected,fname), scores) -> do
  let guessed = VG.maxIndex scores
  putStrLn fname
  printf "Expected: %d Guessed: %d\n" expected guessed
  putStrLn $ "Scores: " ++ show (zip [0..] (VG.toList scores))
  putStrLn $ "Probabilities: " ++ show (zip [0..] (VG.toList (softmax scores)))
  putStrLn ""

0.png
Expected: 0 Guessed: 0
Scores: [(0,9777.892),(1,-7585.3193),(2,-2358.066),(3,-6089.8984),(4,-6311.9727),(5,-6329.25),(6,-2478.122),(7,-4269.2573),(8,-5181.277),(9,-2872.3535)]
Probabilities: [(0,1.0),(1,0.0),(2,0.0),(3,0.0),(4,0.0),(5,0.0),(6,0.0),(7,0.0),(8,0.0),(9,0.0)]

1.png
Expected: 1 Guessed: 1
Scores: [(0,-2165.809),(1,4201.37),(2,-1893.1384),(3,-7467.487),(4,-1335.353),(5,-4800.9795),(6,-416.61957),(7,-2426.936),(8,-503.85828),(9,-5068.9653)]
Probabilities: [(0,0.0),(1,1.0),(2,0.0),(3,0.0),(4,0.0),(5,0.0),(6,0.0),(7,0.0),(8,0.0),(9,0.0)]

2.png
Expected: 2 Guessed: 2
Scores: [(0,-6348.131),(1,-337.71472),(2,7276.1235),(3,-1548.1252),(4,-4145.504),(5,-6463.499),(6,-3721.0723),(7,-1077.4471),(8,-3267.8303),(9,-6660.838)]
Probabilities: [(0,0.0),(1,0.0),(2,1.0),(3,0.0),(4,0.0),(5,0.0),(6,0.0),(7,0.0),(8,0.0),(9,0.0)]

3.png
Expected: 3 Guessed: 3
Scores: [(0,-6894.149),(1,-2907.7124),(2,-1893.7008),(3,7371.175),(4,-8884.441),(5,-4217.8105),(6,-8523.427),(7,-4278.061),(8,-11